# Uploading Batch Input File
 - batch file 보낼 때 너무 많이 보내지 말고 구분해서 보내기

In [1]:
from getpass import getpass

openai_api_key = getpass("OPENAI_API_KEY")

In [2]:
from openai import OpenAI
import json
import pandas as pd

client = OpenAI(api_key=openai_api_key)

In [3]:
batch_input_file = client.files.create(
    file=open("Translate_Mid_batch.jsonl","rb"),
    purpose="batch"
)

print(batch_input_file)

FileObject(id='file-T4UJrJUqWhZTbjhh8pCYB6', bytes=751563, created_at=1739496984, filename='Translate_Mid_batch.jsonl', object='file', purpose='batch', status='processed', status_details=None)


# Create the Batch

In [4]:
# 업로드한 배치 입력 파일 ID 저장
batch_input_file_id = batch_input_file.id
print(batch_input_file_id)

file-T4UJrJUqWhZTbjhh8pCYB6


In [5]:
batch_info = client.batches.create(
    input_file_id = batch_input_file_id,
    endpoint = "/v1/chat/completions", # 챗봇 api
    completion_window = "24h", # 현재는 이 기능만 가능
    metadata = {
        "description": "translate"
    }
)

In [6]:
response_json = json.loads(batch_info.json())
response_output = json.dumps(response_json, ensure_ascii=False, indent = 2)
print(response_output)

{
  "id": "batch_67ae9e1db3dc8190b4b023da52f84aed",
  "completion_window": "24h",
  "created_at": 1739496989,
  "endpoint": "/v1/chat/completions",
  "input_file_id": "file-T4UJrJUqWhZTbjhh8pCYB6",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": null,
  "errors": null,
  "expired_at": null,
  "expires_at": 1739583389,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": {
    "description": "translate"
  },
  "output_file_id": null,
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}


# Checking the Status of Batch
 - validating : the input file is being validated before the batch can begin
 - failed : the input file has failed the validation process
 - in_progress : the input file was successfully validated and the batch is currently being run
 - finalizing : the batch has completed and the results are being prepared
 - completed : the batch has been completed and the results are ready
 - expired : the batch was not able to be completed within the 24-hour time window
 - cancelling : the batch is being cancelled (may take up to 10 minutes)
 - cancelled : the batch was cancelled

- batch_id = 'batch_67ae9e1db3dc8190b4b023da52f84aed' : translate_mid

In [8]:
batch_id = "batch_67ae9e1db3dc8190b4b023da52f84aed"
batch = client.batches.retrieve(batch_id)
batch_json = json.loads(batch.json())
batch_json

{'id': 'batch_67ae9e1db3dc8190b4b023da52f84aed',
 'completion_window': '24h',
 'created_at': 1739496989,
 'endpoint': '/v1/chat/completions',
 'input_file_id': 'file-T4UJrJUqWhZTbjhh8pCYB6',
 'object': 'batch',
 'status': 'completed',
 'cancelled_at': None,
 'cancelling_at': None,
 'completed_at': 1739497425,
 'error_file_id': None,
 'errors': None,
 'expired_at': None,
 'expires_at': 1739583389,
 'failed_at': None,
 'finalizing_at': 1739497387,
 'in_progress_at': 1739496990,
 'metadata': {'description': 'translate'},
 'output_file_id': 'file-BBj8H8RvfthzrwKKTuT8Gx',
 'request_counts': {'completed': 494, 'failed': 0, 'total': 494}}

## 시간 확인

In [9]:
from datetime import datetime
import pytz

kst = pytz.timezone('Asia/Seoul')

# 변환 함수
def convert_timestamp(starttime, endtime):
    st = datetime.utcfromtimestamp(starttime)
    end = datetime.utcfromtimestamp(endtime)
    dt_kst_st = st.replace(tzinfo=pytz.utc).astimezone(kst)
    dt_kst_end = end.replace(tzinfo=pytz.utc).astimezone(kst)

    return f"시작 시간 : {dt_kst_st}, 끝나는 시간 : {dt_kst_end}"

convert_timestamp(starttime = batch_json['created_at'], endtime = batch_json['completed_at'])


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_6420\1843218754.py:8: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  st = datetime.utcfromtimestamp(starttime)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_6420\1843218754.py:9: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  end = datetime.utcfromtimestamp(endtime)


'시작 시간 : 2025-02-14 10:36:29+09:00, 끝나는 시간 : 2025-02-14 10:43:45+09:00'

# Retrieving the Results
 - file-EVCbFfNykgV9HphK2h9wsk

In [10]:
file_response = client.files.content('file-BBj8H8RvfthzrwKKTuT8Gx') # output_file_id 이용
print(file_response.text)

{"id": "batch_req_67ae9fabc6508190a052c363cb0fb223", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "7c3d506f1d56564d40d277acaaa956c3", "body": {"id": "chatcmpl-B0ev6soay2c5uUUn1d7XBNuRtBEOc", "object": "chat.completion", "created": 1739496996, "model": "o3-mini-2025-01-31", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n  \"sub_expression_kor\": \"A: \uc548\ub155, \ub098\ub294 ()\uc57c. ()\uc5d0\uc11c \uc654\uc5b4.\\nB: \ub9cc\ub098\uc11c \ubc18\uac00\uc6cc.\\nA: \uc548\ub155, \ub0b4 \uc774\ub984\uc740 ()\uc57c. ()\uc5d0\uc11c \uc654\uc5b4.\\nB: \uc815\ub9d0 \ubc18\uac00\uc6cc.\"\n}", "refusal": null}, "finish_reason": "stop"}], "usage": {"prompt_tokens": 349, "completion_tokens": 326, "total_tokens": 675, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 256, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_prediction_tokens": 0}}, "service_tier": "defaul

In [11]:
sample_list = file_response.text.split("\n")
sample_list = [item for item in sample_list if item.strip()]
sample_list

['{"id": "batch_req_67ae9fabc6508190a052c363cb0fb223", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "7c3d506f1d56564d40d277acaaa956c3", "body": {"id": "chatcmpl-B0ev6soay2c5uUUn1d7XBNuRtBEOc", "object": "chat.completion", "created": 1739496996, "model": "o3-mini-2025-01-31", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\\n  \\"sub_expression_kor\\": \\"A: \\uc548\\ub155, \\ub098\\ub294 ()\\uc57c. ()\\uc5d0\\uc11c \\uc654\\uc5b4.\\\\nB: \\ub9cc\\ub098\\uc11c \\ubc18\\uac00\\uc6cc.\\\\nA: \\uc548\\ub155, \\ub0b4 \\uc774\\ub984\\uc740 ()\\uc57c. ()\\uc5d0\\uc11c \\uc654\\uc5b4.\\\\nB: \\uc815\\ub9d0 \\ubc18\\uac00\\uc6cc.\\"\\n}", "refusal": null}, "finish_reason": "stop"}], "usage": {"prompt_tokens": 349, "completion_tokens": 326, "total_tokens": 675, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 256, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_pre

In [12]:
sample_output = []

for sample in sample_list:
    for i in range(len(sample_list)):
        sample_json = json.loads(sample)
        content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
        content_json = json.loads(content)
        sample_output.append(f"{i} 번째")
        sample_output.append(content_json)
        #print(json.dumps(content_json, indent=2, ensure_ascii=False))

# print(json.dumps(sample_output))


# batch 결과

In [13]:
for sample in sample_list:
    sample_json = json.loads(sample)
    content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
    content_json = json.loads(content)
    print(json.dumps(content_json, indent=2, ensure_ascii=False))
    break

{
  "sub_expression_kor": "A: 안녕, 나는 ()야. ()에서 왔어.\nB: 만나서 반가워.\nA: 안녕, 내 이름은 ()야. ()에서 왔어.\nB: 정말 반가워."
}


# CSV 파일로 변환

In [14]:
for sample in sample_list:
    sample_json = json.loads(sample)
    content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
    content_json = json.loads(content)
    # print(json.dumps(content_json, indent=2, ensure_ascii=False))
    # break

In [15]:
content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
content_json = json.loads(content)

In [16]:
def JsonToCSV(data:json, file_name:str) -> json:
    data_list = []

    for sample in data:
        # json 변환환
        sample_json = json.loads(sample)
        content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
        content_json = json.loads(content)

        # expression
        sub_expression_kor = content_json["sub_expression_kor"]

        # 데이터 만들기
        csv_data = {
            "Sub Expression (한국어)": sub_expression_kor
        }

        data_list.append(csv_data)

    df = pd.DataFrame(data_list)

    # CSV 저장
    df.to_csv(f"{file_name}.csv", index=False, encoding='utf-8-sig')

    print(f"CSV 변환 완료 파일명: {file_name}.csv")


In [17]:
name = 'Translate_Mid'
JsonToCSV(sample_list, name)

CSV 변환 완료 파일명: Translate_Mid.csv


In [18]:
df = pd.read_csv(f'{name}.csv')
df.head(3)

,Sub Expression (한국어)
0,"A: 안녕, 나는 ()야. ()에서 왔어.\nB: 만나서 반가워.\nA: 안녕, 내..."
1,A: 제일 좋아하는 게 뭐야?\nB: 나는 () 즐겨.\nA: 가장 좋아하는 활동이...
2,A: ~에 대한 계획은 뭐야? B: ~할 계획이야. A: ~에서는 뭐 할 거야? B...


# Cancelling a Batch

In [29]:
# cancel_batch = client.batches.cancel("")
# batch_json = json.loads(cancel_batch.json())
# batch_ouput = json.dumps(batch_json, ensure_ascii=False, indent=2)
# print(batch_ouput)